# О задаче

Наше решение задачи C чемпионата RuCode осени 2023 направления ИИ моей команды (Мой гитхаб: https://github.com/Lex-TS, сокомандника: https://github.com/Lebedkina)

Для решения задачи использовалась модель DecisionTreeClassifier. Результаты прогнозирования записываются в csv файл.

В этой задаче нужно было преобразовать адрес, записанный в одну строку, в несколько отдельных частей, разделяя их по типам в колонки. Мы решили, что эта задача, как и все задачи соревнования по ИИ должна решаться с помощью машинного обучения. Из-за чего выбрали не правильный подход и заполнили только первые 2 столбца из 12 только на закрытом наборе данных, так как оставили эту задачу на самый конец и не успевали обработать другие столбцы.

Как мы поняли позже, эту задачу можно (и нужно) было решать простыми алгоритмическими методами. Достаточно было применить регулярные выражения и, например, алгоритм Ахо—Корасик.

По этой задаче мы получили 16.66 баллов из 100 возможных только на закрытом датасете. По закрытому датасету заняли 8 место из 9, 1 место по этой задаче в дивизионе Начинающие заняла команда с 99.94 баллами

В этом файле представлено решение, которое мы успели сделать до конца проведения соревнования, оно не доделано до конца и распределяет адрес не для всех столбцов данных, так как мы сделали только для двух первых. Этот ноутбук можно воспринимать скорее в качестве черновика, показывающего последовательность нашиих действий при создании решения.

Так как данных было довольно много (15млн строк в обучающей выборке), набор данных делился на половину и каждая половина загружалась отдельно, чтобы не перегружать оперативную память Google Colab.  

#-Загрузка данных

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
# Загрузка основного файла для обучения

# data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/train_100_first.csv', encoding='windows-1251')
# data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['address','municipality', 'municipality_type'], encoding='windows-1251')
# data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['address','settlement', 'settlement_type'], encoding='windows-1251')
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['address'], encoding='windows-1251')

О данных:

*   region_type: NaN = 0
*   settlement_type: NaN = 574892
*   street_type: Nan = 2558837



##--Распределение датасета на два

###---По колонке address

In [ ]:
data_half_1 = pd.DataFrame()

In [ ]:
data_half_2 = pd.DataFrame()

In [ ]:
data_half_1['address'] = data['address'][0:(int(len(data)/2))] #.apply(lambda row: row[0:len(data)/2])

In [ ]:
data_half_2['address'] = data['address'][(int(len(data)/2)):int(len(data))]

In [ ]:
data_half_1.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_address_half_1.csv', index=False)

In [ ]:
data_half_2.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_address_half_2.csv', index=False)

###---По колонке region_type

In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['region_type'], encoding='windows-1251')

In [ ]:
data_rgnTp_half_1 = pd.DataFrame()

In [ ]:
data_rgnTp_half_2 = pd.DataFrame()

In [ ]:
data_rgnTp_half_1['region_type'] = data['region_type'][0:(int(len(data)/2))]

In [ ]:
data_rgnTp_half_2['region_type'] = data['region_type'][(int(len(data)/2)):int(len(data))]

In [ ]:
data_rgnTp_half_1.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_region_half_1.csv', index=False)

In [ ]:
data_rgnTp_half_2.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_region_half_2.csv', index=False)

###---По колонке municipality_type

In [ ]:
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['municipality_type'], encoding='windows-1251')

<ipython-input-25-94dbe0288917>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['municipality_type'], encoding='windows-1251')


In [ ]:
data['municipality_type'][0:(int(len(data)/2))].to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_municipality_half_1.csv', index=False)

In [ ]:
data['municipality_type'][(int(len(data)/2)):int(len(data))].to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_municipality_half_2.csv', index=False)

In [ ]:
del(data)

##--Загрузка половин датасета и работа с ними

###---Работа с region region_type

первая половина

In [ ]:
#Загрузка половины датасета для экономии памяти
data_adrs_half_1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_address_half_1.csv') # первая половина

In [ ]:
region_dict = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dictionaries/uniq_tokens_for_region.csv') #для region

In [ ]:
#Загрузка половины датасета для экономии памяти
data_rgnTp_half_1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_region_half_1.csv') # первая половина

вторая половина

In [ ]:
data_adrs_half_2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_address_half_2.csv') # вторая половина

In [ ]:
data_rgnTp_half_2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_region_half_2.csv') # вторая половина

##--Загружается распарсенные на определенные столбцы данные адресов для создания словарей

In [ ]:
# Загрузка распарсеных на определенные токены адресов
#data_RRtp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_address_RgnRgn_tp.csv') # region region_type
data_MMtp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_address_MncpltMncplt_tp.csv') # municipality	municipality_type

##--Загружаются словари и типы названий для проверки словарей

In [ ]:
# Загрузка адреса и расшифрованной абревиатуры для проверки словаря
data_AdrsRgn_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['address','region_type'], encoding='windows-1251') #для region
region_dict = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dictionaries/uniq_tokens_for_region.csv') #для region

In [ ]:
# Загрузка адреса и расшифрованной абривиатуры для проверки словаря
data_AdrsMncplt_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['address','municipality_type'], encoding='windows-1251') #для municipality

<ipython-input-4-88433e66104d>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data_AdrsMncplt_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['address','municipality_type'], encoding='windows-1251') #для municipality


In [ ]:
municipality_dict = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dictionaries/Копия uniq_tokens_for_municipality.csv') #для municipality

In [ ]:
# Загрузка адреса и расшифрованной абривиатуры для проверки словаря
data_Sttlmnt_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['address','settlement_type'], encoding='windows-1251') #для settlement

In [ ]:
settlement_dict = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dictionaries/uniq_tokens_for_settlementt.csv') #для settlement

In [ ]:
#_location_ Загрузка адреса и расшифрованной абривиатуры для проверки словаря
data_lctn_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['address','location_type'], encoding='windows-1251') #для location
location_dict = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dictionaries/uniq_tokens_for_location.csv') #для location

In [ ]:
#_street_ Загрузка адреса и расшифрованной абривиатуры для проверки словаря
data_Street_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['address','street_type'], encoding='windows-1251') #для street
street_dict = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dictionaries/uniq_tokens_for_street.csv') #для street

#-Просматриваем данные

##--Для адреса

In [ ]:
data

,address
0,а.обл. Еврейская; ; г Биробиджан; ; переулок...
1,А.обл. Еврейская; ; Г Биробиджан; ; пер. Ава...
2,Аобл Еврейская; ; город Биробиджан; ; переул...
3,аобл Еврейская; ; г. Биробиджан; ; переулок ...
4,Аобл Еврейская; ; Г. Биробиджан; ; пер-к Алм...
...,...
15337315,г Севастополь; ; ; ; проспект Гагарина; дом...
15337316,город Севастополь; ; пос Кача; ; ул. Красноа...
15337317,город Севастополь; ; ; территория СТ Солнечн...
15337318,Г. Севастополь; ; ; ; ул Неглинная; дом 2/2; 1


In [ ]:
data.loc[data['region_type'] == "город"]

,address,region_type
125536,город Байконур; ; ; ; микрорайон 5-й; дом 1; 2,город
125537,Г. Байконур; ; ; ; микрорайон 5-й; дом 10; 2,город
125538,Г Байконур; ; ; ; мкр 5-й; дом 12; 2,город
125539,г. Байконур; ; ; ; мкр. 5-й; дом 14; 2,город
125540,город Байконур; ; ; ; мкр 5-й; дом 15; 2,город
...,...,...
15337315,г Севастополь; ; ; ; проспект Гагарина; дом...,город
15337316,город Севастополь; ; пос Кача; ; ул. Красноа...,город
15337317,город Севастополь; ; ; территория СТ Солнечн...,город
15337318,Г. Севастополь; ; ; ; ул Неглинная; дом 2/2; 1,город


In [ ]:
data['region_type'].value_counts(dropna=False)

область               9277195
республика            3117610
край                  2436066
город                  351318
автономный округ       137108
автономная область      18023
Name: region_type, dtype: int64

In [ ]:
data.describe(include='object')

,address,region,region_type,settlement,settlement_type,street,street_type,house,addressTok
count,100,100,100,100,100,100,100,100,100
unique,100,1,1,1,1,16,1,45,100
top,а.обл. Еврейская; ; г Биробиджан; ; переулок...,Еврейская,автономная область,Биробиджан,город,Аремовский,переулок,дом 10,"[а.обл., Еврейская;, ;, г, Биробиджан;, ;, пер..."
freq,1,100,100,100,100,22,100,7,1


In [ ]:
type(data['address'][15337319])

str

In [ ]:
indx = 0
for row in data['settlement_type']:
  if (str(row).find("железнодорожный")+1):
    print(data.iloc[indx])
  indx+=1

address            обл Амурская; р-н Магдагачинский; ж/д бл-ст. К...
settlement                                              Красная Падь
settlement_type                             железнодорожный блокпост
Name: 1778993, dtype: object
address            Обл. Амурская; р-н. Магдагачинский; ж/д бл-ст....
settlement                                              Красная Падь
settlement_type                             железнодорожный блокпост
Name: 1778994, dtype: object
address            Обл. Амурская; р-н Магдагачинский; ж/д бл-ст К...
settlement                                              Красная Падь
settlement_type                             железнодорожный блокпост
Name: 1778995, dtype: object
address            Обл. Амурская; район Магдагачинский; ж/д бл-ст...
settlement                                              Красная Падь
settlement_type                             железнодорожный блокпост
Name: 1778996, dtype: object
address            область Амурская; р-н. Магдагачинский

In [ ]:
pd.Series(data['settlement_type'].unique()).to_frame()

,0
0,город
1,село
2,NaN
3,станция
4,поселок
...,...
57,поселок разъезда
58,железнодорожный остановочный пункт
59,железнодорожный путевой пост
60,планировочный район


##--Для region region_type

In [ ]:
type(data_RRtp)

pandas.core.frame.DataFrame

In [ ]:
data_RRtp

,token_region
0,а.обл. Еврейская
1,А.обл. Еврейская
2,Аобл Еврейская
3,аобл Еврейская
4,Аобл Еврейская
...,...
15337315,г Севастополь
15337316,город Севастополь
15337317,город Севастополь
15337318,Г. Севастополь


##--Для municipality	municipality_type

In [ ]:
data_MMtp

,token_municipality
0,р-н. Биробиджанский
1,р-н. Биробиджанский
2,р-н Биробиджанский
3,район Биробиджанский
4,р-н Биробиджанский
...,...
9090165,р-н Симферопольский
9090166,район Симферопольский
9090167,г Алушта
9090168,город Алушта


In [ ]:
type(data_MMtp)

pandas.core.frame.DataFrame

In [ ]:
data_MMtp.squeeze().unique()

array([nan, ' р-н. Биробиджанский', ' р-н Биробиджанский', ...,
       ' г. Евпатория', ' Г. Судак', ' Г. Евпатория'], dtype=object)

тер 47 км шоссе СПб-Пески

#-Работаем со всем адресом

Разбиваем адрес на токены, разделитель ';'

##--Для region region_type

In [ ]:
#street = data['address'].apply(lambda row: row.split(';')[0])
tokens_RRtp = data['address'].apply(lambda row: row.split(';')[0])

In [ ]:
tokens_RRtp

0            а.обл. Еврейская
1            А.обл. Еврейская
2              Аобл Еврейская
3              аобл Еврейская
4              Аобл Еврейская
                  ...        
15337315        г Севастополь
15337316    город Севастополь
15337317    город Севастополь
15337318       Г. Севастополь
15337319       Г. Севастополь
Name: address, Length: 15337320, dtype: object

In [ ]:
#street.to_frame()
tokens_RRtp = tokens_RRtp.to_frame()

In [ ]:
tokens_RRtp.rename({'address': 'token_region'}, axis=1, inplace=True)

In [ ]:
tokens_RRtp

,token_region
0,а.обл. Еврейская
1,А.обл. Еврейская
2,Аобл Еврейская
3,аобл Еврейская
4,Аобл Еврейская
...,...
15337315,г Севастополь
15337316,город Севастополь
15337317,город Севастополь
15337318,Г. Севастополь


##--Для municipality	municipality_type

In [ ]:
#street = data['address'].apply(lambda row: row.split(';')[0])
tokens_MMtp = data['address'].apply(lambda row: row.split(';')[1])

In [ ]:
#street.to_frame()
tokens_MMtp = tokens_MMtp.to_frame()

In [ ]:
tokens_MMtp.rename({'address': 'token_municipality'}, axis=1, inplace=True)

In [ ]:
tokens_MMtp

,token_municipality
0,
1,
2,
3,
4,
...,...
15337315,
15337316,
15337317,
15337318,


##--Сохраняем разделенную по ';' часть адреса

###---Часть region, region_type

In [ ]:
#street.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_address_RgnRgn_tp.csv',index=False)
tokens_RRtp.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_address_RgnRgn_tp.csv', index=False)

###---Часть municipality	municipality_type

In [ ]:
#street.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_address_RgnRgn_tp.csv',index=False)
tokens_MMtp.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_address_MncpltMncplt_tp.csv', index=False)

#-Добавляет функцию обработки аббревиатуры  

In [ ]:
def normalize_abbrv(abbrv):
  new_abbrv = abbrv.lower()
  new_abbrv = new_abbrv.replace('.', '')
  return(new_abbrv)

#-Создаются словари аббревиатур для каждого столбца

##--Работаем с region, region_type (data_RRtp)

In [ ]:
data_RRtp['token_region'] = data_RRtp['token_region'].apply(lambda row: row.split())

In [ ]:
data_RRtp

,token_region
0,"[а.обл., Еврейская]"
1,"[А.обл., Еврейская]"
2,"[Аобл, Еврейская]"
3,"[аобл, Еврейская]"
4,"[Аобл, Еврейская]"
...,...
15337315,"[г, Севастополь]"
15337316,"[город, Севастополь]"
15337317,"[город, Севастополь]"
15337318,"[Г., Севастополь]"


In [ ]:
tok1_RRtp = pd.DataFrame()

In [ ]:
tok1_RRtp['token'] = data_RRtp['token_region'].apply(lambda row: row[0])

In [ ]:
tok1_RRtp['token'].value_counts()

область       1856519
обл.          1855556
обл           1855492
Обл.          1854906
Обл           1854722
край           609773
Край.          609251
край.          608804
Край           608238
рес            446240
респ.          445831
республика     445589
Респ.          445234
Респ           445035
респ           444942
рес.           444739
город           70596
г.              70448
г               70177
Г               70115
Г.              69982
а.окр.          34461
аокр            34332
А.окр.          34202
Аокр            34113
А.обл.           4648
Аобл             4502
а.обл.           4441
аобл             4432
Name: token, dtype: int64

In [ ]:
tok1_RRtp['token'] = tok1_RRtp['token'].apply(lambda row: normalize_abbrv(row))

In [ ]:
tok1_RRtp['token'].value_counts(dropna= False)

обл           7420676
край          2436066
область       1856519
респ          1781042
рес            890979
республика     445589
г              280722
аокр           137108
город           70596
аобл            18023
Name: token, dtype: int64

In [ ]:
unic_toks = tok1_RRtp['token'].unique()

In [ ]:
unic_toks = pd.Series(unic_toks,name = 'abbreviation').to_frame()

In [ ]:
unic_toks

,abbreviation
0,аобл
1,аокр
2,город
3,г
4,край
5,обл
6,область
7,рес
8,республика
9,респ


In [ ]:
unic_toks.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/uniq_tokens_for_region.csv', index=False)

##--Работаем с municipality	municipality_type (data_MMtp)

nan = 6247150

In [ ]:
data_MMtp['token_municipality'] = data_MMtp['token_municipality'].apply(lambda row: str(row).split())

In [ ]:
data_MMtp

,token_municipality
0,"[р-н., Биробиджанский]"
1,"[р-н., Биробиджанский]"
2,"[р-н, Биробиджанский]"
3,"[район, Биробиджанский]"
4,"[р-н, Биробиджанский]"
...,...
9090165,"[р-н, Симферопольский]"
9090166,"[район, Симферопольский]"
9090167,"[г, Алушта]"
9090168,"[город, Алушта]"


In [ ]:
tok1_MMtp = pd.DataFrame()

In [ ]:
tok1_MMtp['token'] = data_MMtp['token_municipality'].apply(lambda row: row[0])

In [ ]:
tok1_MMtp

,token
0,р-н.
1,р-н.
2,р-н
3,район
4,р-н
...,...
9090165,р-н
9090166,район
9090167,г
9090168,город


In [ ]:
tok1_MMtp['token'].value_counts().to_frame()

,token
р-н,2823729
р-н.,2821101
район,2820082
город,103521
г,103479
г.,103222
Г,103071
Г.,102807
У.,11711
у.,11701


In [ ]:
tok1_MMtp['token'] = tok1_MMtp['token'].apply(lambda row: normalize_abbrv(row))

In [ ]:
tok1_MMtp['token'].value_counts()

р-н          5644830
район        2820082
г             412579
город         103521
у              46610
улус           11509
сп             11420
пос             6737
п               6588
с/п             5607
пгт             5469
рп              5169
пос-е           3376
поселение       3364
м               1351
с                505
м-в              355
зона             336
массив           330
село             247
сс                94
сельсовет         34
с/с               18
с/мо              18
смо               14
тер                4
пос-к              2
терр               1
Name: token, dtype: int64

In [ ]:
unic_toks = tok1_MMtp['token'].unique()

In [ ]:
unic_toks

array(['р-н', 'район', 'г', 'город', 'сп', 'с/п', 'улус', 'у', 'сс',
       'сельсовет', 'с/с', 'пгт', 'пос-к', 'пос', 'п', 'с/мо', 'смо', 'м',
       'зона', 'м-в', 'массив', 'тер', 'терр', 'рп', 'с', 'село', 'пос-е',
       'поселение'], dtype=object)

In [ ]:
unic_toks = pd.Series(unic_toks,name = 'abbreviation').to_frame()

In [ ]:
unic_toks

,abbreviation
0,р-н
1,район
2,г
3,город
4,сп
5,с/п
6,улус
7,у
8,сс
9,сельсовет


In [ ]:
unic_toks.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/uniq_tokens_for_municipality.csv', index=False)

#-Проверяем правильно ли заполненен словарь

##--Для region region_type

In [ ]:
data_AdrsRgn_tp['region_type'] = data_AdrsRgn_tp['region_type'].apply(lambda row: row.split())

In [ ]:
indexes = list([])

In [ ]:
for i in range(len(data_AdrsRgn_tp['region_type'])):
  if len(data_AdrsRgn_tp['region_type'][i])<2:
    indexes.append(i)
    #data_AdrsRgn_tp=data_AdrsRgn_tp.drop([i])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
data_AdrsRgn_tp

,address,region_type
0,а.обл. Еврейская; ; г Биробиджан; ; переулок...,"[автономная, область]"
1,А.обл. Еврейская; ; Г Биробиджан; ; пер. Ава...,"[автономная, область]"
2,Аобл Еврейская; ; город Биробиджан; ; переул...,"[автономная, область]"
3,аобл Еврейская; ; г. Биробиджан; ; переулок ...,"[автономная, область]"
4,Аобл Еврейская; ; Г. Биробиджан; ; пер-к Алм...,"[автономная, область]"
...,...,...
15337315,г Севастополь; ; ; ; проспект Гагарина; дом...,[город]
15337316,город Севастополь; ; пос Кача; ; ул. Красноа...,[город]
15337317,город Севастополь; ; ; территория СТ Солнечн...,[город]
15337318,Г. Севастополь; ; ; ; ул Неглинная; дом 2/2; 1,[город]


In [ ]:
data_AdrsRgn_tp=data_AdrsRgn_tp.drop(indexes)

In [ ]:
data_AdrsRgn_tp

,address,region_type
0,а.обл. Еврейская; ; г Биробиджан; ; переулок...,"[автономная, область]"
1,А.обл. Еврейская; ; Г Биробиджан; ; пер. Ава...,"[автономная, область]"
2,Аобл Еврейская; ; город Биробиджан; ; переул...,"[автономная, область]"
3,аобл Еврейская; ; г. Биробиджан; ; переулок ...,"[автономная, область]"
4,Аобл Еврейская; ; Г. Биробиджан; ; пер-к Алм...,"[автономная, область]"
...,...,...
15282027,аокр Ямало-Ненецкий; район Надымский; пос-к Яг...,"[автономный, округ]"
15282028,а.окр. Ямало-Ненецкий; район Тазовский; п. Таз...,"[автономный, округ]"
15282029,Аокр Ямало-Ненецкий; р-н. Пуровский; Г. Тарко-...,"[автономный, округ]"
15282030,Аокр Ямало-Ненецкий; р-н Надымский; село Кутоп...,"[автономный, округ]"


In [ ]:
data_AdrsRgn_tp['region_type'].value_counts().to_frame()

,region_type
"[автономный, округ]",137108
"[автономная, область]",18023


##--Для municipality municipality_type

###---Ищем выражения в municipality_type длиннее 2 слов

In [ ]:
data_AdrsMncplt_tp['municipality_type'] = data_AdrsMncplt_tp['municipality_type'].apply(lambda row: str(row).split())

In [ ]:
data_AdrsMncplt_tp

,address,municipality_type
0,а.обл. Еврейская; ; г Биробиджан; ; переулок...,NaN
1,А.обл. Еврейская; ; Г Биробиджан; ; пер. Ава...,NaN
2,Аобл Еврейская; ; город Биробиджан; ; переул...,NaN
3,аобл Еврейская; ; г. Биробиджан; ; переулок ...,NaN
4,Аобл Еврейская; ; Г. Биробиджан; ; пер-к Алм...,NaN
...,...,...
15337315,г Севастополь; ; ; ; проспект Гагарина; дом...,NaN
15337316,город Севастополь; ; пос Кача; ; ул. Красноа...,NaN
15337317,город Севастополь; ; ; территория СТ Солнечн...,NaN
15337318,Г. Севастополь; ; ; ; ул Неглинная; дом 2/2; 1,NaN


In [ ]:
indexes = list([])

In [ ]:
len(str(data_AdrsMncplt_tp['municipality_type'][0]).split())

1

In [ ]:
for i in range(len(data_AdrsMncplt_tp['municipality_type'])):
  if len(str(data_AdrsMncplt_tp['municipality_type'][i]).split())<2:
    indexes.append(i)

In [ ]:
data_AdrsMncplt_tp

,address,municipality_type
0,а.обл. Еврейская; ; г Биробиджан; ; переулок...,NaN
1,А.обл. Еврейская; ; Г Биробиджан; ; пер. Ава...,NaN
2,Аобл Еврейская; ; город Биробиджан; ; переул...,NaN
3,аобл Еврейская; ; г. Биробиджан; ; переулок ...,NaN
4,Аобл Еврейская; ; Г. Биробиджан; ; пер-к Алм...,NaN
...,...,...
15337315,г Севастополь; ; ; ; проспект Гагарина; дом...,NaN
15337316,город Севастополь; ; пос Кача; ; ул. Красноа...,NaN
15337317,город Севастополь; ; ; территория СТ Солнечн...,NaN
15337318,Г. Севастополь; ; ; ; ул Неглинная; дом 2/2; 1,NaN


In [ ]:
data_AdrsMncplt_tp=data_AdrsMncplt_tp.drop(indexes)

In [ ]:
data_AdrsMncplt_tp

,address,municipality_type
9568447,Респ. Алтай; сп Ининское; с Чуйозы; ; ул. Г.М...,"[сельское, поселение]"
9568544,респ Алтай; с.п. Ининское; село Чуйозы; ; ул ...,"[сельское, поселение]"
9568557,рес Алтай; сп Ининское; с. Чуйозы; ; улица Г....,"[сельское, поселение]"
9569041,рес. Алтай; сп Ининское; с. Чуйозы; ; ул Г.М....,"[сельское, поселение]"
9730530,рес. Ингушетия; с/п Джейрах; село Джейрах; ; ...,"[сельское, поселение]"
...,...,...
14709198,область Тверская; с.п. Центральное; д. Новые М...,"[сельское, поселение]"
14709201,обл. Тверская; с.п. Алферовское; Д. Иванково; ...,"[сельское, поселение]"
15255701,аокр Ханты-Мансийский; сп Солнечный; территори...,"[сельское, поселение]"
15261618,Аокр Ханты-Мансийский; с/п Солнечный; терр Про...,"[сельское, поселение]"


In [ ]:
data_AdrsMncplt_tp['municipality_type'].value_counts().to_frame()

,municipality_type
сельское поселение,17027
поселок городского типа,5469
рабочий поселок,5169
сельское муницип.образование,32


###---После нахождения нужных выражений добавляем их в существующий словарь

Добавим в словарь выражения, которые длиннее 1 слова

In [ ]:
def tokList_to_str(tok):
  new_str = tok[0]
  if len(tok)>1:
    for i in range(1,len(tok)):
      new_str = new_str+' '+tok[i]
  return new_str

In [ ]:
data_AdrsMncplt_tp['municipality_type'] = data_AdrsMncplt_tp['municipality_type'].apply(lambda row: tokList_to_str(row))

In [ ]:
data_AdrsMncplt_tp

,address,municipality_type
9568447,Респ. Алтай; сп Ининское; с Чуйозы; ; ул. Г.М...,сельское поселение
9568544,респ Алтай; с.п. Ининское; село Чуйозы; ; ул ...,сельское поселение
9568557,рес Алтай; сп Ининское; с. Чуйозы; ; улица Г....,сельское поселение
9569041,рес. Алтай; сп Ининское; с. Чуйозы; ; ул Г.М....,сельское поселение
9730530,рес. Ингушетия; с/п Джейрах; село Джейрах; ; ...,сельское поселение
...,...,...
14709198,область Тверская; с.п. Центральное; д. Новые М...,сельское поселение
14709201,обл. Тверская; с.п. Алферовское; Д. Иванково; ...,сельское поселение
15255701,аокр Ханты-Мансийский; сп Солнечный; территори...,сельское поселение
15261618,Аокр Ханты-Мансийский; с/п Солнечный; терр Про...,сельское поселение


In [ ]:
additional_toks = pd.Series(data_AdrsMncplt_tp['municipality_type']).unique()

In [ ]:
pd.Series(additional_toks).to_frame()

,0
0,сельское поселение
1,поселок городского типа
2,сельское муницип.образование
3,рабочий поселок


In [ ]:
additional_toks = pd.Series(additional_toks,name = 'abbreviation').to_frame()

In [ ]:
additional_toks

,abbreviation
0,сельское поселение
1,поселок городского типа
2,сельское муницип.образование
3,рабочий поселок


In [ ]:
municipality_dict = pd.concat([municipality_dict,additional_toks],ignore_index=True)

In [ ]:
municipality_dict

,abbreviation
0,р-н
1,район
2,г
3,город
4,сп
5,с/п
6,улус
7,у
8,сс
9,сельсовет


###---Сохраняем измененный словарь

In [ ]:
municipality_dict.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/uniq_tokens_for_municipality.csv', index=False)

In [ ]:
additional_toks = pd.Series(data_AdrsMncplt_tp['municipality_type']).unique()

##--Функция проверки словаря и добавления новых выражений

In [ ]:
def check_corr_of_dict(data_address_col_type, type_of_col, dict_for_col, path_to_dict):
  #Ищем выражения в municipality_type длиннее 2 слов
  indexes = list([])
  #data_address_col_type должен быть пронумерован четкой последовательностью для корректной работы цикла
  for i in range(len(data_address_col_type[type_of_col])):
    if len(str(data_address_col_type[type_of_col][i]).split())<2:
      indexes.append(i)

  data_address_col_type=data_address_col_type.drop(indexes)
  print("Удалены строки длиной в одно слово")

  #Добавим в словарь выражения, которые длиннее 1 слова
  data_address_col_type[type_of_col] = data_address_col_type[type_of_col].apply(lambda row: normalize_abbrv(row))
  additional_toks = pd.Series(data_address_col_type[type_of_col]).unique()
  print("_debug: создан additional_toks")
  if len(additional_toks)>0:
    print("_debug: сработал if")
    additional_toks = pd.Series(additional_toks, name = 'abbreviation').to_frame()
    print("_debug: additional_toks переделан в DataFrame")
    dict_for_col = pd.concat([dict_for_col, additional_toks],ignore_index=True)
    del(additional_toks)
    dict_for_col['length'] = dict_for_col['abbreviation'].apply(lambda row: len(row.split()))
    dict_for_col = dict_for_col.sort_values(by=['length'], ascending=False, ignore_index=True ).drop(columns='length')
    print("В словарь добавлены новые выражения")
  else:
    print("Выражения длиннее 1 слова не найдены")

  #Сохраняем измененный словарь
  dict_for_col.to_csv(path_to_dict, index=False)
  print("Словарь сохранен")

In [ ]:
check_corr_of_dict(data_AdrsMncplt_tp, 'municipality_type', municipality_dict, '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dicts/uniq_tokens_for_municipality.csv')

Удалены строки длиной в одно слово
_debug: создан additional_toks
_debug: сработал if
_debug: additional_toks переделан в DataFrame
В словарь добавлены новые выражения
Словарь сохранен


In [ ]:
check_corr_of_dict(data_Sttlmnt_tp, 'settlement_type', settlement_dict, '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dicts/uniq_tokens_for_settlementt.csv')

Удалены строки длиной в одно слово
_debug: создан additional_toks
_debug: сработал if
_debug: additional_toks переделан в DataFrame
В словарь добавлены новые выражения
Словарь сохранен


In [ ]:
check_corr_of_dict(data_lctn_tp, 'location_type', location_dict, '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dicts/uniq_tokens_for_location.csv')

Удалены строки длиной в одно слово
_debug: создан additional_toks
_debug: сработал if
_debug: additional_toks переделан в DataFrame
В словарь добавлены новые выражения
Словарь сохранен


In [ ]:
check_corr_of_dict(data_Street_tp, 'street_type', street_dict, '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dicts/uniq_tokens_for_street.csv')

Удалены строки длиной в одно слово
_debug: создан additional_toks
_debug: сработал if
_debug: additional_toks переделан в DataFrame
В словарь добавлены новые выражения
Словарь сохранен


#-One-hot кодирование типов аббревиатур для классификации

In [ ]:
data_Rgntp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['region_type'], encoding='windows-1251')

##--Определяем частоту аббревиатур и создаем словари (слово | код)

In [ ]:
def abbrv_class(full_abbrv_uniq):
  i = 0
  abbrv_class = [0]
  for full_abbrv in full_abbrv_uniq:
    i += 1
    abbrv_class.append(i)
  return abbrv_class

In [ ]:
def makeAndSave_full_abbrv_dict(data_col_type, col_type, save_path):
  col_type_full_abbrv_dict = pd.DataFrame()
  col_type_full_abbrv_dict['full_abbreviation'] = {0: 'NaN'}
  col_type_full_abbrv_dict = pd.concat([col_type_full_abbrv_dict['full_abbreviation'], pd.Series(pd.Series(data_col_type[col_type]).unique())], ignore_index = True).rename("full_abbreviation").to_frame()
  col_type_full_abbrv_dict['class_code'] =  pd.Series(abbrv_class(pd.Series(data_col_type[col_type]).unique())).to_frame()
  col_type_full_abbrv_dict.to_csv(save_path, index=False)

In [ ]:
makeAndSave_full_abbrv_dict(data_Rgntp, 'region_type', '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/type_dict_class/dict_class_region.csv')

In [ ]:
del(data_Rgntp)
data_col_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['municipality_type'], encoding='windows-1251')

<ipython-input-114-8fa195d7c318>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data_col_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['municipality_type'], encoding='windows-1251')


In [ ]:
makeAndSave_full_abbrv_dict(data_col_tp, 'municipality_type', '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/type_dict_class/dict_class_municipality.csv')

In [ ]:
del(data_col_tp)
data_col_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['settlement_type'], encoding='windows-1251')

In [ ]:
makeAndSave_full_abbrv_dict(data_col_tp, 'settlement_type', '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/type_dict_class/dict_class_settlement.csv')

In [ ]:
del(data_col_tp)
data_col_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['location_type'], encoding='windows-1251')

In [ ]:
makeAndSave_full_abbrv_dict(data_col_tp, 'location_type', '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/type_dict_class/dict_class_location.csv')

In [ ]:
del(data_col_tp)
data_col_tp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/addresses-train.csv', usecols=['street_type'], encoding='windows-1251')

In [ ]:
makeAndSave_full_abbrv_dict(data_col_tp, 'street_type', '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/type_dict_class/dict_class_street.csv')

##--Фукнция для перевода из номера класса в полную аббревиатуру

In [ ]:
def (pred)

#-Распределение выражений по колонкам

##--Данные для примера

In [ ]:
sample_dict_orig2 = ['аобл','сельское','сельское муницип образование', 'автономная область', 'город','железнодророжный разъезд','желенодорожный сборочный пункт']

In [ ]:
sample_dict_orig2 = pd.Series(sample_dict_orig2, name = 'abbreviation').to_frame()

In [ ]:
sample_dict_orig2['length'] = sample_dict_orig2['abbreviation'].apply(lambda row: len(row.split()))

In [ ]:
sample_dict_orig2

,abbreviation,length
0,аобл,1
1,сельское,1
2,сельское муницип образование,3
3,автономная область,2
4,город,1
5,железнодророжный разъезд,2
6,желенодорожный сборочный пункт,3


In [ ]:
sample_dict_orig2 = sample_dict_orig2.sort_values(by=['length'], ascending=False, ignore_index=True ).drop(columns='length')

In [ ]:
sample_dict_orig2

,abbreviation
0,сельское муницип образование
1,желенодорожный сборочный пункт
2,автономная область
3,железнодророжный разъезд
4,аобл
5,сельское
6,город


In [ ]:
sample_address = {'address': ['  ', 'КСТ СНТ Железнодорожников 15 Сельское муницип. образование', 'Сельское. КСТ СНТ Железнодорожников 15', '  ',
                              'КСТ СНТ Железнодорожников 15 Сельское', '  ', 'Сельское муницип. образование Еврейская',
                              'Горно-Алтайская Сельское Карачаево-Черкесская муницип. Еврейская образование ', '  ',
                              'Биробиджан Город', 'г Биробиджан', '  ', 'железнодорожный проезд 11 Москва', '12 Тверь проезд железнодорожный',
                              'железндорожный разъезд 30 Омск', 'А.Обл Алтайская']}

In [ ]:
sample_address_full = {'address': ['а.обл. Еврейская;  ; г Биробиджан;  ; переулок Авангардный; дом 1; 2', 'КСТ СНТ Железнодорожников 15 Сельское муницип. образование;  ; г Биробиджан;  ; переулок Авангардный; дом 2; 2',
                              'Сельское. КСТ СНТ Железнодорожников 15;  ; г Биробиджан;  ; переулок Авангардный; дом 2; 2', 'КСТ СНТ Железнодорожников 15 Сельское;  ; г Биробиджан;  ; переулок Авангардный; дом 1; 1',
                              'Сельское муницип. образование Еврейская;  ; г Биробиджан;  ; переулок Авангардный; дом 3; 2',
                              'Горно-Алтайская Сельское Карачаево-Черкесская муницип. Еврейская образование;  ; г Биробиджан;  ; переулок Авангардный; дом 4; 2',
                              'Биробиджан Город;  ; г Биробиджан;  ; переулок Авангардный; дом 4; 2', 'г Биробиджан;  ; г Биробиджан;  ; переулок Авангардный; дом 3; 2',
                              'железнодорожный проезд 11 Москва;  ; г Биробиджан;  ; переулок Авангардный; дом 1; 2', '12 Тверь проезд железнодорожный;  ; г Биробиджан;  ; переулок Авангардный; дом 3; 2',
                              'железндорожный разъезд 30 Омск;  ; г Биробиджан;  ; переулок Авангардный; дом 2; 2', 'А.Обл Алтайская;  ; г Биробиджан;  ; переулок Авангардный; дом 1; 2']}

In [ ]:
sample_address2 = {'address': ['КСТ СНТ Железнодорожников 15 Сельское муницип. образование', 'Сельское. КСТ СНТ Железнодорожников 15',
                              'КСТ СНТ Железнодорожников 15 Сельское', 'Сельское муницип. образование Еврейская',
                              'Горно-Алтайская Сельское Карачаево-Черкесская муницип. Еврейская образование ']}

In [ ]:
sample_address_full = pd.DataFrame(sample_address_full)

In [ ]:
sample_address_full

,address
0,а.обл. Еврейская; ; г Биробиджан; ; переулок...
1,КСТ СНТ Железнодорожников 15 Сельское муницип....
2,Сельское. КСТ СНТ Железнодорожников 15; ; г Б...
3,КСТ СНТ Железнодорожников 15 Сельское; ; г Би...
4,Сельское муницип. образование Еврейская; ; г ...
5,Горно-Алтайская Сельское Карачаево-Черкесская ...
6,Биробиджан Город; ; г Биробиджан; ; переулок...
7,г Биробиджан; ; г Биробиджан; ; переулок Ава...
8,железнодорожный проезд 11 Москва; ; г Биробид...
9,12 Тверь проезд железнодорожный; ; г Биробидж...


##--Функции для нахождения аббревиатур

In [ ]:
def find_abbrv(dict_for_col, data_address):
  if (len(data_address)>0):
    for i in range(len(data_address)):
      data_address[i] = normalize_abbrv(data_address[i])
    ids_inner_list = []
    for word in dict_for_col['abbreviation']:
      word = word.split()
      word_len = len(word)
      if word_len>1:
        for i in range(len(data_address)):
          for tok in word:
            if tok == data_address[i]:
              ids_inner_list.append(i)
              break
        if len(ids_inner_list)>0:
          if len(ids_inner_list)<word_len:
            ids_inner_list = []
          elif len(ids_inner_list)==word_len:
            return ids_inner_list
      else:
        for i in range(len(data_address)):
          if word[0] == data_address[i]:
            ids_inner_list.append(i)
            break
        if len(ids_inner_list)>0:
          return ids_inner_list
    if len(ids_inner_list)==0:
      print('Error: abbreviation not found')
      print("__In line: "+str(data_address))
  else:
    return [np.nan]
  return ids_inner_list

In [ ]:
def split_address(tokenized_address,indx):
  abbrv_lst = []
  name_lst = []
  for i in range(len(tokenized_address)):
    if i in indx:
      abbrv_lst.append(tokenized_address[i])
    else:
      name_lst.append(tokenized_address[i])
  abbrv_line = ''
  name_line = ''
  if len(abbrv_lst) > 0:
    abbrv_line = abbrv_lst[0]
    for word_id in range(1,len(abbrv_lst)):
      abbrv_line = abbrv_line + ' ' + abbrv_lst[word_id]
  else:
    abbrv_line = np.nan
  if len(name_lst) > 0:
    name_line = name_lst[0]
    for word_id in range(1,len(name_lst)):
      name_line = name_line + ' ' + name_lst[word_id]
  else:
    name_line = np.nan
  del(abbrv_lst)
  del(name_lst)
  return list([name_line, abbrv_line])

In [ ]:
def split_cols_and_save(df, dict_for_col, col_name, col_num, path_for_save, row_with_abbrv = True):
  df = df['address'].apply(lambda row: row.split(';')).to_frame()
  if row_with_abbrv:
    df[col_name] = df['address'].apply(lambda row: split_address(row[col_num].split(), find_abbrv(dict_for_col, row[col_num].split())))
    df[col_name + '_type'] = df[col_name].apply(lambda row: row[1])
    df[col_name] = df[col_name].apply(lambda row: row[0])
    df.to_csv(path_for_save, columns=[col_name, col_name +'_type'], index=False)
  else:
    df[col_name] = df['address'].apply(lambda row: row[col_num]).to_frame()
    df.to_csv(path_for_save, columns=[col_name], index=False)

##--Пример работы функций

In [ ]:
split_cols_and_save(sample_address_full, sample_dict_orig2, 'region', 0, 'region.csv')

Error: abbreviation not found
__In line: ['г', 'биробиджан']
Error: abbreviation not found
__In line: ['железнодорожный', 'проезд', '11', 'москва']
Error: abbreviation not found
__In line: ['12', 'тверь', 'проезд', 'железнодорожный']
Error: abbreviation not found
__In line: ['железндорожный', 'разъезд', '30', 'омск']


In [ ]:
split_cols_and_save(sample_address_full, sample_dict_orig2, 'house', 5, 'house.csv', False)

In [ ]:
new_df = pd.read_csv('house.csv')
new_df

,house
0,дом 1
1,дом 2
2,дом 2
3,дом 1
4,дом 3
5,дом 4
6,дом 4
7,дом 3
8,дом 1
9,дом 3


##--Поиск аббревиатур в основном датасете

первая половина датасета

In [ ]:
split_cols_and_save(data_adrs_half_1, region_dict, 'region', 0, '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/region_half_1.csv')

In [ ]:
new_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/region_half_1.csv')
new_df

вторая половина датасета

In [ ]:
split_cols_and_save(data_adrs_half_2, region_dict, 'region', 0, '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/region_half_2.csv')

In [ ]:
new_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/region_half_2.csv')
new_df

,region,region_type
0,Бурятия,Респ.
1,Бурятия,рес
2,Бурятия,респ.
3,Бурятия,республика
4,Бурятия,республика
...,...,...
7668655,Севастополь,г
7668656,Севастополь,город
7668657,Севастополь,город
7668658,Севастополь,Г.


для municipality

первая половина датасета

In [ ]:
municipality_dict = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dictionaries/uniq_tokens_for_municipality.csv') #для municipality

In [ ]:
split_cols_and_save(data_adrs_half_1, municipality_dict, 'municipality', 1, '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/municipality_half_1.csv')

In [ ]:
del(data_adrs_half_1)

вторая половина датасета

In [ ]:
split_cols_and_save(data_adrs_half_2, municipality_dict, 'municipality', 1, '/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/municipality_half_2.csv')

In [ ]:
del(data_adrs_half_2)

In [ ]:
#split_cols_and_save(sample_address_full, sample_dict_orig2, 'house', 5, 'house.csv', False)

#-Обучаем модель для предсказания класса аббревиатуры

In [ ]:
#reg_tp_clasess = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/type_dict_class/dict_class_region.csv')

In [ ]:
munic_tp_clasess = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/type_dict_class/dict_class_municipality.csv')

In [ ]:
munic_tp_clasess

,full_abbreviation,class_code
0,NaN,0
1,NaN,1
2,район,2
3,город,3
4,сельское поселение,4
5,улус,5
6,сельсовет,6
7,поселок городского типа,7
8,поселок,8
9,сельское муницип.образование,9


In [ ]:
splitted_adrsCol_regionTp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/region_half_1.csv', usecols=['region_type']) # первая половина

In [ ]:
splitted_adrsCol_regionTp_2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/region_half_2.csv', usecols=['region_type']) # вторая половина

для municipality_type

In [ ]:
splitted_adrsCol_municTp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/municipality_half_1.csv', usecols=['municipality_type']) # первая половина

In [ ]:
splitted_adrsCol_municTp_2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/splitted_found_cols/municipality_half_2.csv', usecols=['municipality_type']) # вторая половина

In [ ]:
splitted_adrsCol_municTp_2

,municipality_type
0,район
1,район
2,р-н
3,р-н.
4,р-н
...,...
7668655,NaN
7668656,NaN
7668657,NaN
7668658,NaN


In [ ]:
#Загрузка половины датасета для экономии памяти
data_mncpltTp_half_1 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_municipality_half_1.csv') # первая половина

In [ ]:
data_mncpltTp_half_2 = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/half_parts_address/data_municipality_half_2.csv') # вторая половина

In [ ]:
data_mncpltTp_half_1 = pd.concat([data_mncpltTp_half_1['municipality_type'], data_mncpltTp_half_2['municipality_type']], ignore_index = True).rename("municipality_type").to_frame()

In [ ]:
del(data_mncpltTp_half_2)

In [ ]:
splitted_adrsCol_municTp = pd.concat([splitted_adrsCol_municTp['municipality_type'], splitted_adrsCol_municTp_2['municipality_type']], ignore_index = True).rename("municipality_type").to_frame()

In [ ]:
splitted_adrsCol_municTp

,municipality_type
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
15337315,NaN
15337316,NaN
15337317,NaN
15337318,NaN


In [ ]:
del(splitted_adrsCol_municTp_2)

In [ ]:
munic_tp_clasess['full_abbreviation'][4]

'сельское поселение'

In [ ]:
def to_class(df_row, class_dict):
  for i in range(len(class_dict)):
    if df_row == class_dict['full_abbreviation'][i]:
      #df_row = class_dict['class_code'][i]
      return class_dict['class_code'][i]
  return 0

In [ ]:
data_mncpltTp_half_1['municipality_type'] = data_mncpltTp_half_1['municipality_type'].apply(lambda row: to_class(row, munic_tp_clasess)).to_frame()

In [ ]:
data_mncpltTp_half_1

,municipality_type
0,0
1,0
2,0
3,0
4,0
...,...
15337315,0
15337316,0
15337317,0
15337318,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

In [ ]:
from sklearn.model_selection import train_test_split

y = data_mncpltTp_half_1['municipality_type']
X = count_vect.fit_transform(splitted_adrsCol_municTp['municipality_type'].apply(lambda row: str(row)))

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=12)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_train

3629277     0
9931106     5
487731      0
12742931    2
3894710     2
           ..
6855921     2
9850109     2
7595398     2
3905179     2
7812939     2
Name: municipality_type, Length: 10736124, dtype: int64

In [ ]:
X_train

<10736124x19 sparse matrix of type '<class 'numpy.int64'>'
	with 6448903 stored elements in Compressed Sparse Row format>

In [ ]:
#X_train_counts = count_vect.fit_transform(X_train)
X_train.shape

(10736124, 19)

In [ ]:
#X_test_counts = count_vect.fit_transform(X_test)
X_test.shape

(4601196, 19)

# Модель DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

DecisionTreeClassifier()

In [ ]:
y_pred2 = dt.predict(X_test)

In [ ]:
acc2 = accuracy_score(y_test,y_pred2)

In [ ]:
acc2

0.968599685820817

In [ ]:
del(X_train)
del(y_train)
del(X_test)
del(y_test)
#del(X_train_counts)
#del(X_test_counts)
del(splitted_adrsCol_regionTp)
del(data_rgnTp_half_1)

In [ ]:
private_df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/private/addresses-close.csv', encoding='windows-1251')

In [ ]:
private_df

,address
0,А.обл. Еврейская; ; г Биробиджан; ; переулок...
1,аобл Еврейская; ; г. Биробиджан; ; пер-к Алм...
2,Аобл Еврейская; ; г Биробиджан; ; пер-к Анга...
3,а.обл. Еврейская; ; г. Биробиджан; ; переуло...
4,Аобл Еврейская; ; Г. Биробиджан; ; переулок ...
...,...
5112415,Г Севастополь; ; ; территория тсн ТСН СТ Авт...
5112416,г Севастополь; ; ; территория тсн ТСН СТ Ово...
5112417,г. Севастополь; ; ; тер. ТСН ДНТ Волга; ; д...
5112418,город Севастополь; ; ; территория тсн Коллек...


In [ ]:
region_dict = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/dictionaries/uniq_tokens_for_region.csv') #для region

In [ ]:
private_df['address'][0:(int(len(private_df)/2))].to_csv('privateData_address_half_1.csv', index=False)

In [ ]:
private_df['address'][int(len(private_df)/2):int(len(private_df))].to_csv('privateData_address_half_2.csv', index=False)

In [ ]:
del(private_df)

In [ ]:
private_df_half_1 = pd.read_csv('privateData_address_half_1.csv') #первая половина закрытого датасета

In [ ]:
split_cols_and_save(private_df_half_1, region_dict, 'region', 0, 'region_half_1.csv')

In [ ]:
del(private_df_half_1)

In [ ]:
private_df_half_2 = pd.read_csv('privateData_address_half_2.csv') #вторая половина закрытого датасета

In [ ]:
split_cols_and_save(private_df_half_2, region_dict, 'region', 0, 'region_half_2.csv')

In [ ]:
del(private_df_half_2)

In [ ]:
split_cols_and_save(sample_address_full, sample_dict_orig2, 'house', 5, 'house.csv', False)

In [ ]:
#Загрузка половины датасета для экономии памяти
dataPrvt_rgnTp_half_1 = pd.read_csv('region_half_1.csv') # первая половина

In [ ]:
dataPrvt_rgnTp_half_2 = pd.read_csv('region_half_2.csv') # вторая половина

In [ ]:
dataPrvt_rgnTp_half_1 = pd.concat([dataPrvt_rgnTp_half_1, dataPrvt_rgnTp_half_2], ignore_index = True)

In [ ]:
del(dataPrvt_rgnTp_half_2)

In [ ]:
dataPrvt_rgnTp_half_1.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/private_cols_splitted/privateData_regionSplitted.csv', index=False)

In [ ]:
def to_abbrv(pred, class_dict):
  pred_list = []
  for pred_id in range(len(pred)):
    for i in range(len(class_dict)):
      if pred[pred_id] == class_dict['class_code'][i]:
        #df_row = class_dict['class_code'][i]
        pred_list.append(class_dict['full_abbreviation'][i])
        break
  return pred_list

In [ ]:
X_counts = count_vect.fit_transform(dataPrvt_rgnTp_half_1['region_type'])
X_counts.shape

(5112420, 10)

In [ ]:
y_pred2 = dt.predict(X_counts)

#Сохранение предсказаний в csv

In [ ]:
new_data = pd.DataFrame()

In [ ]:
new_data['region_type'] = pd.Series(to_abbrv(y_pred2, reg_tp_clasess)).to_frame()

In [ ]:
answer = pd.DataFrame(columns=['region', 'region_type', 'municipality', 'municipality_type', 'settlement',
                        'settlement_type', 'location', 'location_type', 'street', 'street_type', 'house', 'source'])

In [ ]:
answer

,region,region_type,municipality,municipality_type,settlement,settlement_type,location,location_type,street,street_type,house,source


In [ ]:
new_data

,region_type
0,область
1,область
2,область
3,республика
4,область
...,...
2300593,республика
2300594,область
2300595,область
2300596,город


In [ ]:
answer['region']=dataPrvt_rgnTp_half_1['region']

In [ ]:
answer['region_type']=new_data['region_type']

In [ ]:
answer.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/answers/answer_region.csv', index=False)

In [ ]:
answer.to_csv('/content/gdrive/My Drive/Colab Notebooks/dataset_task_C/answers/answer_region_enc_windows-1251.csv', index=False, encoding='windows-1251')

In [ ]:
smpl = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/dataset/mars-sample_submission-regr.csv')
smpl